In [12]:
import numpy as np
import os
import matplotlib.pyplot as plt
import os
import time
import glob
# DATA = os.getenv('DEXNET_DATA')
# print(DATA)

In [14]:
arrays = {}
datapoint = '_05590.npz'                #Enter datapoint here (anywhere between _00000 and _06728, there are a total of 6.7 million datapoints i.e, 6728 x 1000). Change this value to just '.npz' to train on entire dataset
for filename in os.listdir('/home/firefly/dexnet/dexnet_2/dexnet_2_tensor/tensors/'):
    # data = '_05590.npz'                
    for npz_file_number in range (2):   #Depending on number of examples just change the range.
        data = '_{0:05}.npz'.format(npz_file_number) 
      
        if filename.endswith(data):
            arrays[filename.replace('.npz', '')] = np.load( '/home/firefly/dexnet/dexnet_2/dexnet_2_tensor/tensors/' + filename)
            #print(arrays)
    
    
    

features = {}
for array in arrays:
    f = arrays[array]
    feature = f['arr_0.npy']
    features[array] = feature
print(features.keys())



dict_keys(['robust_ferrari_canny_00000', 'robust_ferrari_canny_00001', 'hand_poses_00001', 'depth_ims_tf_table_00001', 'hand_poses_00000', 'depth_ims_tf_table_00000'])


print(data)

In [126]:
# #Inputs to feed into CNN

#-----------x-------------------
search_key1 = 'depth_ims_tf_table'
aligned_imgs_x1 = np.array([val for key, val in features.items() if search_key1 in key])
print(np.shape(aligned_imgs_x1))

search_key2 = 'hand_poses'
gripper_depths_x2 = np.array([val for key, val in features.items() if search_key2 in key])
gripper_depths_x2 = gripper_depths_x2[:, :, 2]
print(np.shape(gripper_depths_x2))


#------------y------------------
search_key3 = 'robust_ferrari_canny'
grasp_metrics_y = np.array([val for key, val in features.items() if search_key3 in key])
print(np.shape(grasp_metrics_y))



(2, 1000, 32, 32, 1)
(2, 1000)
(2, 1000)


In [127]:
aligned_imgs=np.zeros([aligned_imgs_x1.shape[0], aligned_imgs_x1.shape[1], aligned_imgs_x1.shape[2], aligned_imgs_x1.shape[3]], aligned_imgs_x1.shape[4])
print(aligned_imgs.shape)
for i in range(len(aligned_imgs_x1)):
    aligned_imgs[i, :, :, :, :] = aligned_imgs_x1[i]

TypeError: Cannot interpret '1' as a data type

In [48]:
#******************One hot encoding***********************

# print(grasp_metrics[2])
def one_hot_encoding(grasp_metrics):
    for i in range(1000):
        if(grasp_metrics[i] > 0.002):  #threshold value is 0.002 readme
            grasp_metrics[i] = 1
        else:
            grasp_metrics[i] = 0
    return grasp_metrics

grasp_metrics = one_hot_encoding(grasp_metrics)

# print(grasp_metrics)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [49]:
import tensorflow as tf
import math
import random

# example of defining the discriminator model
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Lambda
from tensorflow.nn import local_response_normalization
from keras.utils.vis_utils import plot_model

from keras.layers.merge import concatenate
from keras.models import Model
# from tensorflow.keras import layers
from keras.layers import Dense, Input


#TODO: Initialize weights in layers according to the paper https://arxiv.org/pdf/1703.09312.pdf
def getGraspQualityVariable():
    input = Input(shape=(32, 32, 1), name="img")
    x1 = Conv2D(filters=64, kernel_size=7, activation='relu')(input)
    x2 = Conv2D(filters=64, kernel_size=5, activation='relu')(x1)
    x3 = Lambda(local_response_normalization)(x2)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=2)(x3)
    x5 = Conv2D(filters=64, kernel_size=3, activation='relu')(x4)
    # x6 = Dropout(0.3)(x5)
    x6 = Conv2D(filters=64, kernel_size=3, activation='relu')(x5)
    x7 = Lambda(local_response_normalization)(x6)
    x8 = Flatten()(x7)
    x9 = Dense(1024, activation='relu')(x8)
    
    # plot_model(model, to_file='GraspQualityModel_plot.png', show_shapes=True, show_layer_names=True)
    return x9, input
    
getGraspQualityVariable()


def getPointcloudModel():
    input = Input(shape=(1), name="z")
    x = Dense(16, input_dim=1, activation='relu')(input)
    return x, input


from keras.layers.merge import concatenate
def getDexnet2Model():
    grasp_model, input_1 = getGraspQualityVariable()
    pc_model, input_2 = getPointcloudModel()

    out = Dense(1024, activation='relu')

    # x = layers.concatenate([grasp_model, pc_model])
    merge = concatenate([grasp_model, pc_model])
    out_1 = Dense(1024, activation='relu')(merge)
    out_2 = Dense(2, activation='softmax')(out_1)
    model = Model(inputs=[input_1, input_2], outputs=out_2)
    plot_model(model, to_file='Dexnet2_plot.png', show_shapes=True, show_layer_names=True)
    return model


In [50]:
first_model = getDexnet2Model()


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [51]:
first_model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 26, 26, 64)   3200        img[0][0]                        
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 22, 22, 64)   102464      conv2d_16[0][0]                  
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 22, 22, 64)   0           conv2d_17[0][0]                  
____________________________________________________________________________________________

In [52]:
first_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


In [53]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='/home/firefly/dexnet/dexnet2_git/Weights/second_model.weights.best.hdf5', verbose = 1, save_best_only=True)
x_train = [aligned_imgs, gripper_depths]                                   #Check model.summary() in previous section and check model architecture in paper
y_train = grasp_metrics                                                    #valued between [0, 1] grasp robustness for the given grasp
first_model.fit(x_train,
          y_train,
          batch_size=16,                                                   #worth ecperimenting
          epochs=5,                                      
          validation_split=0.25,                                            #Decide on a number
          callbacks=[checkpointer])        

                                #CHange the filepath of the checkpointer varianble to store different version of weights.


Epoch 1/10
47/47 [==============================] - 3s 52ms/step - loss: 0.4035 - accuracy: 0.8497 - val_loss: 0.2669 - val_accuracy: 0.9320

Epoch 00001: val_loss improved from inf to 0.26694, saving model to /home/firefly/dexnet/dexnet2_git/Weights/second_model.weights.best.hdf5
Epoch 2/10
47/47 [==============================] - 2s 50ms/step - loss: 0.3206 - accuracy: 0.9135 - val_loss: 0.2523 - val_accuracy: 0.9320

Epoch 00002: val_loss improved from 0.26694 to 0.25226, saving model to /home/firefly/dexnet/dexnet2_git/Weights/second_model.weights.best.hdf5
Epoch 3/10
47/47 [==============================] - 2s 50ms/step - loss: 0.2786 - accuracy: 0.9258 - val_loss: 0.2533 - val_accuracy: 0.9320

Epoch 00003: val_loss did not improve from 0.25226
Epoch 4/10
47/47 [==============================] - 2s 49ms/step - loss: 0.3146 - accuracy: 0.9080 - val_loss: 0.2480 - val_accuracy: 0.9320

Epoch 00004: val_loss improved from 0.25226 to 0.24800, saving model to /home/firefly/dexnet/dexn

In [20]:
first_model.predict([aligned_imgs[290:350],gripper_depths[290:350]])

array([[1.00000000e+00, 1.57385473e-18],
       [1.00000000e+00, 9.76254430e-15],
       [1.00000000e+00, 4.45226313e-29],
       [1.00000000e+00, 8.44479286e-20],
       [1.00000000e+00, 1.93126236e-20],
       [1.00000000e+00, 6.48304130e-31],
       [5.87066129e-09, 1.00000000e+00],
       [1.00000000e+00, 4.43832960e-19],
       [1.00000000e+00, 2.50874625e-14],
       [1.00000000e+00, 9.92938340e-29],
       [1.00000000e+00, 6.03239298e-19],
       [1.00000000e+00, 2.84128119e-21],
       [1.00000000e+00, 1.05569032e-31],
       [2.02726120e-08, 1.00000000e+00],
       [1.00000000e+00, 4.37048438e-20],
       [1.00000000e+00, 2.50651388e-13],
       [1.00000000e+00, 8.49463526e-30],
       [1.00000000e+00, 6.00779469e-20],
       [1.00000000e+00, 6.42472595e-21],
       [1.00000000e+00, 9.09340154e-32],
       [2.52236476e-09, 1.00000000e+00],
       [1.00000000e+00, 5.52493641e-20],
       [1.00000000e+00, 2.65539527e-14],
       [1.00000000e+00, 3.75436209e-29],
       [1.000000